<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/99_GenerarVisualizacionesPDF_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
Sistema de Generación de PDFs con Control de Peso
===================================================================

Versión mejorada con las siguientes características:
- Control de peso máximo por PDF (15MB configurable)
- Generación automática de múltiples partes cuando se supera el límite
- Selección de modelos específicos para reprocesar
- Mantiene sistema de checkpoints y detección automática
- Trazabilidad completa con TQDM

Autor: Jesús L.
Universidad: Universidad Oberta de Catalunya (UOC)


"""

'\nSistema de Generación de PDFs con Control de Peso\n===================================================================\n\nVersión mejorada con las siguientes características:\n- Control de peso máximo por PDF (15MB configurable)\n- Generación automática de múltiples partes cuando se supera el límite\n- Selección de modelos específicos para reprocesar\n- Mantiene sistema de checkpoints y detección automática\n- Trazabilidad completa con TQDM\n\nAutor: Jesús L.\nUniversidad: Universidad Oberta de Catalunya (UOC)\n\n\n'

In [2]:
# =============================================================================
# CONFIGURACIÓN INICIAL
# =============================================================================

print("=" * 70)
print("SISTEMA DE GENERACIÓN DE PDFs CON CONTROL DE PESO - TFM")
print("=" * 70)
print("\nPaso 1/4: Montando Google Drive...")

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

print("Google Drive montado correctamente\n")

SISTEMA DE GENERACIÓN DE PDFs CON CONTROL DE PESO - TFM

Paso 1/4: Montando Google Drive...
Mounted at /content/drive
Google Drive montado correctamente



In [3]:
# =============================================================================
# INSTALACIÓN DE DEPENDENCIAS
# =============================================================================

print("Paso 2/4: Instalando dependencias...")
import subprocess
import sys

resultado = subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "img2pdf", "Pillow", "tqdm"],
    capture_output=True
)

if resultado.returncode == 0:
    print("Dependencias instaladas: img2pdf, Pillow, tqdm\n")
else:
    print("Error instalando dependencias")
    print(resultado.stderr.decode())

Paso 2/4: Instalando dependencias...
Dependencias instaladas: img2pdf, Pillow, tqdm



In [4]:
# =============================================================================
# CÓDIGO PRINCIPAL
# =============================================================================

print("Paso 3/4: Cargando módulos...\n")

import json
import logging
import re
import pickle
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Set, Tuple, Optional
from dataclasses import dataclass, field

try:
    import img2pdf
    from PIL import Image
    from tqdm.auto import tqdm
except ImportError as e:
    print(f"ERROR: No se pudieron importar dependencias: {e}")
    print("Ejecute: !pip install img2pdf Pillow tqdm")
    raise

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

Paso 3/4: Cargando módulos...



In [5]:
@dataclass
class ConfiguracionPDF:
    """
    Configuración para la generación de PDFs con control de peso.

    Attributes:
        peso_maximo_mb: Peso máximo permitido por PDF en megabytes
        modelos_reprocesar: Lista de modelos a reprocesar. Si es None, procesa todos
        prefijo_partes: Prefijo para nombrar las partes cuando se divide un PDF
    """
    peso_maximo_mb: float = 15.0
    modelos_reprocesar: Optional[List[str]] = None
    prefijo_partes: str = "parte"

    def debe_procesar_modelo(self, nombre_config: str) -> bool:
        """
        Determina si una configuración debe procesarse según los modelos seleccionados.

        Args:
            nombre_config: Nombre de la configuración a evaluar

        Returns:
            True si debe procesarse, False en caso contrario
        """
        if self.modelos_reprocesar is None:
            return True

        nombre_lower = nombre_config.lower()
        return any(modelo.lower() in nombre_lower for modelo in self.modelos_reprocesar)


@dataclass
class EstadisticasGeneracion:
    """Estadísticas del proceso de generación."""
    visualizaciones_omitidas: int = 0
    archivos_no_encontrados: int = 0
    extensiones_invalidas: int = 0
    alpha_warnings: int = 0
    pdfs_generados: int = 0
    pdfs_con_partes: int = 0
    total_partes_generadas: int = 0
    errores: int = 0
    total_imagenes_procesadas: int = 0


class AlphaWarningCounter(logging.Handler):
    """Handler para contar warnings de alpha channel de img2pdf."""

    def __init__(self):
        super().__init__()
        self.count = 0

    def emit(self, record):
        if 'alpha' in record.getMessage().lower():
            self.count += 1

In [9]:
class GeneradorPDFVisualizaciones:
    """
    Generador de PDFs con control de peso y detección automática de progreso.

    Características principales:
    - Control de peso máximo por PDF (15MB por defecto)
    - Generación automática de múltiples partes
    - Selección de modelos específicos para reprocesar
    - Detecta PDFs ya existentes
    - Sistema de checkpoints cada 5 PDFs
    - Continúa automáticamente si se interrumpe
    - Trazabilidad completa con TQDM
    """

    PATRON_UMBRAL_DECIMAL = re.compile(r'_t0[._]\d+$')
    PATRON_SENSIBILIDAD_ESPECIFICA = re.compile(
        r'_(baja|media|alta|maxima)_sensibilidad_t0[._]\d+$'
    )
    EXTENSIONES_VALIDAS = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}
    CHECKPOINT_INTERVALO = 5

    def __init__(
        self,
        ruta_indice: str,
        directorio_salida: str,
        config_pdf: Optional[ConfiguracionPDF] = None
    ) -> None:
        """
        Inicializa el generador con detección de PDFs existentes.

        Args:
            ruta_indice: Ruta al archivo JSON del índice maestro
            directorio_salida: Directorio donde se guardarán los PDFs
            config_pdf: Configuración para generación de PDFs
        """
        self.ruta_indice = Path(ruta_indice)
        self.directorio_salida = Path(directorio_salida)
        self.checkpoint_file = self.directorio_salida / '.checkpoint.pkl'
        self.config_pdf = config_pdf or ConfiguracionPDF()

        if not self.ruta_indice.exists():
            raise FileNotFoundError(f"Índice maestro no encontrado: {self.ruta_indice}")

        self.directorio_salida.mkdir(parents=True, exist_ok=True)

        self.indice_maestro: Optional[Dict] = None
        self.agrupaciones: defaultdict = defaultdict(list)

        self.estadisticas = EstadisticasGeneracion()

        # Configurar contador de alpha warnings
        self.alpha_counter = AlphaWarningCounter()
        self.alpha_counter.setLevel(logging.WARNING)
        img2pdf_logger = logging.getLogger('img2pdf')
        img2pdf_logger.addHandler(self.alpha_counter)

        # Cargar checkpoint O detectar PDFs existentes
        self.configs_completadas = self._cargar_o_detectar_progreso()

        logger.info("Inicializado GeneradorPDFVisualizaciones v2.0.0")
        logger.info(f"  Índice: {self.ruta_indice}")
        logger.info(f"  Salida: {self.directorio_salida}")
        logger.info(f"  Peso máximo por PDF: {self.config_pdf.peso_maximo_mb} MB")

        if self.config_pdf.modelos_reprocesar:
            logger.info(f"  Modelos a procesar: {', '.join(self.config_pdf.modelos_reprocesar)}")
        else:
            logger.info(f"  Modelos a procesar: TODOS")

        if self.configs_completadas:
            logger.info(f"  PDFs detectados: {len(self.configs_completadas)}")

    def _cargar_o_detectar_progreso(self) -> Set[str]:
        """
        Carga checkpoint O detecta PDFs existentes en el directorio.

        Permite continuar sin perder progreso de ejecuciones previas.
        Detecta tanto PDFs simples como PDFs con múltiples partes.
        """
        # Intentar cargar checkpoint
        if self.checkpoint_file.exists():
            try:
                with open(self.checkpoint_file, 'rb') as f:
                    checkpoint_data = pickle.load(f)
                    if isinstance(checkpoint_data, set):
                        logger.info("Checkpoint encontrado")
                        return checkpoint_data
            except Exception as e:
                logger.warning(f"Checkpoint corrupto: {e}")

        # Si no hay checkpoint, detectar PDFs existentes
        logger.info("Detectando PDFs existentes en directorio...")
        pdfs_existentes = set()

        try:
            for pdf_file in self.directorio_salida.glob('*.pdf'):
                config_name = self._extraer_nombre_base_config(pdf_file.stem)
                pdfs_existentes.add(config_name)

            if pdfs_existentes:
                logger.info(f"Detectados {len(pdfs_existentes)} configuraciones con PDFs existentes")
                logger.info("  Estos NO serán regenerados")
                self._guardar_checkpoint_inicial(pdfs_existentes)
            else:
                logger.info("  No se encontraron PDFs previos")

            return pdfs_existentes

        except Exception as e:
            logger.warning(f"Error detectando PDFs: {e}")
            return set()

    def _extraer_nombre_base_config(self, nombre_archivo: str) -> str:
        """
        Extrae el nombre base de configuración eliminando sufijos de partes.

        Ejemplos:
            "mask2former_base_coco_media_parte_1" -> "mask2former_base_coco_media"
            "sam_default" -> "sam_default"

        Args:
            nombre_archivo: Nombre del archivo PDF (sin extensión)

        Returns:
            Nombre base de la configuración
        """
        patron = re.compile(rf'_{self.config_pdf.prefijo_partes}_\d+$')
        return patron.sub('', nombre_archivo)

    def _guardar_checkpoint_inicial(self, configs: Set[str]) -> None:
        """Guarda checkpoint inicial con PDFs detectados."""
        try:
            with open(self.checkpoint_file, 'wb') as f:
                pickle.dump(configs, f, protocol=pickle.HIGHEST_PROTOCOL)
            logger.info("Checkpoint inicial creado")
        except Exception as e:
            logger.warning(f"No se pudo crear checkpoint: {e}")

    def _guardar_checkpoint(self) -> None:
        """Guarda checkpoint periódicamente."""
        try:
            with open(self.checkpoint_file, 'wb') as f:
                pickle.dump(self.configs_completadas, f, protocol=pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            logger.warning(f"Error guardando checkpoint: {e}")

    def _eliminar_checkpoint(self) -> None:
        """Elimina checkpoint al finalizar."""
        if self.checkpoint_file.exists():
            try:
                self.checkpoint_file.unlink()
                logger.info("Checkpoint eliminado (proceso completado)")
            except Exception as e:
                logger.warning(f"Error eliminando checkpoint: {e}")

    def cargar_indice(self) -> None:
        """Carga el índice maestro desde JSON."""
        logger.info("Cargando índice maestro...")
        try:
            with open(self.ruta_indice, 'r', encoding='utf-8') as archivo:
                self.indice_maestro = json.load(archivo)
            logger.info(f"  Índice cargado: {len(self.indice_maestro)} fotografías")
        except Exception as error:
            logger.error(f"Error cargando índice: {error}")
            raise

    def extraer_configuraciones(self) -> Set[str]:
        """
        Extrae todas las configuraciones únicas del índice maestro.

        Aplica filtrado por modelos si está configurado.
        """
        logger.info("Extrayendo configuraciones...")

        if not self.indice_maestro:
            logger.error("Índice maestro no cargado")
            return set()

        todas_configs = set()

        for fotografia_data in self.indice_maestro.values():
            for modelo, configuraciones in fotografia_data.items():
                if not isinstance(configuraciones, dict):
                    continue

                for config in configuraciones.keys():
                    todas_configs.add(config)

        # Aplicar filtro de modelos si está configurado
        if self.config_pdf.modelos_reprocesar:
            configs_filtradas = {
                cfg for cfg in todas_configs
                if self.config_pdf.debe_procesar_modelo(cfg)
            }
            logger.info(f"  Configuraciones totales: {len(todas_configs)}")
            logger.info(f"  Configuraciones filtradas: {len(configs_filtradas)}")
            return configs_filtradas
        else:
            logger.info(f"  Configuraciones encontradas: {len(todas_configs)}")
            return todas_configs

    def agrupar_por_configuracion(self) -> None:
        """Agrupa visualizaciones por configuración."""
        logger.info("Agrupando visualizaciones por configuración...")

        if not self.indice_maestro:
            logger.error("Índice maestro no cargado")
            return

        for foto_id, datos_foto in self.indice_maestro.items():
            # El índice maestro tiene estructura:
            # foto_id -> {'modelos_disponibles': {codigo_config: {...}}}

            modelos_disponibles = datos_foto.get('modelos_disponibles', {})

            if not isinstance(modelos_disponibles, dict):
                logger.warning(f"Estructura incorrecta en foto {foto_id}")
                continue

            for codigo_config, datos_config in modelos_disponibles.items():
                # Aplicar filtro de modelos
                if not self.config_pdf.debe_procesar_modelo(codigo_config):
                    self.estadisticas.visualizaciones_omitidas += 1
                    continue

                # Extraer ruta de visualización
                ruta_viz = datos_config.get('ruta_visualizacion')

                # Validación crítica: debe ser string o None
                if ruta_viz is None:
                    continue

                if not isinstance(ruta_viz, str):
                    logger.warning(
                        f"Ruta de visualización no es string para {codigo_config} "
                        f"en {foto_id}: tipo={type(ruta_viz).__name__}"
                    )
                    self.estadisticas.archivos_no_encontrados += 1
                    continue

                ruta_path = Path(ruta_viz)

                if not ruta_path.exists():
                    logger.warning(f"Archivo no encontrado: {ruta_viz}")
                    self.estadisticas.archivos_no_encontrados += 1
                    continue

                if ruta_path.suffix not in self.EXTENSIONES_VALIDAS:
                    logger.debug(f"Extensión inválida: {ruta_path.suffix} en {ruta_viz}")
                    self.estadisticas.extensiones_invalidas += 1
                    continue

                self.agrupaciones[codigo_config].append(str(ruta_path))

        logger.info(f"  Configuraciones válidas agrupadas: {len(self.agrupaciones)}")
        logger.info(f"  Visualizaciones omitidas (filtro): {self.estadisticas.visualizaciones_omitidas}")
        logger.info(f"  Archivos no encontrados: {self.estadisticas.archivos_no_encontrados}")
        logger.info(f"  Extensiones inválidas: {self.estadisticas.extensiones_invalidas}")

    def ordenar_visualizaciones(self, rutas: List[str]) -> List[str]:
        """
        Ordena las rutas por número de fotografía extraído del nombre.

        Args:
            rutas: Lista de rutas a ordenar

        Returns:
            Lista ordenada de rutas
        """
        patron = re.compile(r'foto(\d{3})')

        def extraer_numero(ruta: str) -> int:
            match = patron.search(Path(ruta).name)
            return int(match.group(1)) if match else 999999

        return sorted(rutas, key=extraer_numero)

    def _calcular_peso_estimado(self, rutas_imagenes: List[str]) -> float:
        """
        Calcula el peso estimado del PDF resultante.

        Se basa en el peso de las imágenes más un overhead del 5%.

        Args:
            rutas_imagenes: Lista de rutas a las imágenes

        Returns:
            Peso estimado en megabytes
        """
        peso_total_bytes = sum(Path(ruta).stat().st_size for ruta in rutas_imagenes)
        # Agregar overhead del 5% para metadata PDF
        peso_estimado_bytes = peso_total_bytes * 1.05
        return peso_estimado_bytes / (1024 * 1024)

    def _dividir_imagenes_por_peso(
        self,
        rutas_imagenes: List[str]
    ) -> List[List[str]]:
        """
        Divide una lista de imágenes en grupos que no excedan el peso máximo.

        Args:
            rutas_imagenes: Lista ordenada de rutas a imágenes

        Returns:
            Lista de listas, cada sublista representa un grupo de imágenes
            que no excede el peso máximo
        """
        grupos = []
        grupo_actual = []
        peso_grupo_actual = 0.0

        for ruta in rutas_imagenes:
            peso_imagen_mb = Path(ruta).stat().st_size / (1024 * 1024)
            peso_estimado = peso_imagen_mb * 1.05  # Overhead 5%

            # Si una sola imagen excede el límite, crear grupo con solo esa imagen
            if peso_estimado >= self.config_pdf.peso_maximo_mb:
                if grupo_actual:
                    grupos.append(grupo_actual)
                    grupo_actual = []
                    peso_grupo_actual = 0.0
                grupos.append([ruta])
                logger.warning(
                    f"Imagen individual excede límite: {Path(ruta).name} "
                    f"({peso_estimado:.2f} MB)"
                )
                continue

            # Si agregar esta imagen excede el límite, iniciar nuevo grupo
            if peso_grupo_actual + peso_estimado > self.config_pdf.peso_maximo_mb:
                if grupo_actual:
                    grupos.append(grupo_actual)
                grupo_actual = [ruta]
                peso_grupo_actual = peso_estimado
            else:
                grupo_actual.append(ruta)
                peso_grupo_actual += peso_estimado

        # Agregar el último grupo si tiene imágenes
        if grupo_actual:
            grupos.append(grupo_actual)

        return grupos

    def generar_pdf(
        self,
        nombre_config: str,
        rutas_imagenes: List[str],
        pbar: Optional[tqdm] = None
    ) -> List[Path]:
        """
        Genera PDF o múltiples PDFs según el peso.

        Args:
            nombre_config: Nombre de la configuración
            rutas_imagenes: Lista ordenada de rutas a imágenes
            pbar: Barra de progreso opcional

        Returns:
            Lista de rutas a los PDFs generados
        """
        # Normalizar nombre de configuración
        config_normalizado = self._normalizar_nombre(nombre_config)

        # Calcular peso estimado total
        peso_estimado_total = self._calcular_peso_estimado(rutas_imagenes)

        # Decidir si es necesario dividir
        if peso_estimado_total <= self.config_pdf.peso_maximo_mb:
            # Generar PDF único
            return [self._generar_pdf_unico(
                config_normalizado,
                rutas_imagenes,
                pbar,
                peso_estimado=peso_estimado_total
            )]
        else:
            # Generar múltiples PDFs
            return self._generar_pdfs_multiples(
                config_normalizado,
                rutas_imagenes,
                pbar,
                peso_estimado_total=peso_estimado_total
            )

    def _normalizar_nombre(self, nombre: str) -> str:
        """Normaliza el nombre de configuración para nombre de archivo."""
        nombre = nombre.replace('/', '_').replace('\\', '_')

        # Normalizar separadores decimales en umbrales
        if self.PATRON_UMBRAL_DECIMAL.search(nombre):
            nombre = re.sub(r'_t0\.', '_t0_', nombre)

        if self.PATRON_SENSIBILIDAD_ESPECIFICA.search(nombre):
            nombre = re.sub(r'sensibilidad_t0\.', 'sensibilidad_t0_', nombre)

        return nombre

    def _generar_pdf_unico(
        self,
        nombre_config: str,
        rutas_imagenes: List[str],
        pbar: Optional[tqdm],
        peso_estimado: float
    ) -> Path:
        """
        Genera un único PDF.

        Args:
            nombre_config: Nombre normalizado de la configuración
            rutas_imagenes: Lista de rutas a imágenes
            pbar: Barra de progreso opcional
            peso_estimado: Peso estimado en MB

        Returns:
            Ruta al PDF generado
        """
        nombre_pdf = f"{nombre_config}.pdf"
        ruta_pdf = self.directorio_salida / nombre_pdf

        if pbar:
            pbar.set_description(f"PDF: {nombre_pdf[:45]}")

        try:
            count_before = self.alpha_counter.count
            pdf_bytes = img2pdf.convert(rutas_imagenes)
            count_after = self.alpha_counter.count

            alpha_warnings_este_pdf = count_after - count_before
            if alpha_warnings_este_pdf > 0:
                self.estadisticas.alpha_warnings += alpha_warnings_este_pdf

            with open(ruta_pdf, 'wb') as archivo_pdf:
                archivo_pdf.write(pdf_bytes)

            tamanio_mb = ruta_pdf.stat().st_size / (1024 * 1024)
            self.estadisticas.pdfs_generados += 1
            self.estadisticas.total_imagenes_procesadas += len(rutas_imagenes)

            if pbar:
                pbar.set_postfix({
                    'MB': f'{tamanio_mb:.1f}',
                    'imgs': len(rutas_imagenes),
                    'alpha': self.estadisticas.alpha_warnings
                })

            return ruta_pdf

        except Exception as error:
            self.estadisticas.errores += 1
            logger.error(f"Error en {nombre_pdf}: {error}")
            raise

    def _generar_pdfs_multiples(
        self,
        nombre_config: str,
        rutas_imagenes: List[str],
        pbar: Optional[tqdm],
        peso_estimado_total: float
    ) -> List[Path]:
        """
        Genera múltiples PDFs cuando el peso excede el límite.

        Args:
            nombre_config: Nombre normalizado de la configuración
            rutas_imagenes: Lista completa de rutas a imágenes
            pbar: Barra de progreso opcional
            peso_estimado_total: Peso estimado total en MB

        Returns:
            Lista de rutas a los PDFs generados
        """
        logger.info(
            f"Dividiendo PDF {nombre_config}: "
            f"{peso_estimado_total:.1f} MB > {self.config_pdf.peso_maximo_mb} MB"
        )

        # Dividir imágenes en grupos
        grupos = self._dividir_imagenes_por_peso(rutas_imagenes)

        pdfs_generados = []

        for idx, grupo in enumerate(grupos, start=1):
            nombre_parte = (
                f"{nombre_config}_"
                f"{self.config_pdf.prefijo_partes}_{idx}.pdf"
            )
            ruta_pdf = self.directorio_salida / nombre_parte

            if pbar:
                pbar.set_description(
                    f"PDF: {nombre_parte[:40]} ({idx}/{len(grupos)})"
                )

            try:
                count_before = self.alpha_counter.count
                pdf_bytes = img2pdf.convert(grupo)
                count_after = self.alpha_counter.count

                alpha_warnings_este_pdf = count_after - count_before
                if alpha_warnings_este_pdf > 0:
                    self.estadisticas.alpha_warnings += alpha_warnings_este_pdf

                with open(ruta_pdf, 'wb') as archivo_pdf:
                    archivo_pdf.write(pdf_bytes)

                tamanio_mb = ruta_pdf.stat().st_size / (1024 * 1024)
                self.estadisticas.total_imagenes_procesadas += len(grupo)

                pdfs_generados.append(ruta_pdf)

                if pbar:
                    pbar.set_postfix({
                        'parte': f'{idx}/{len(grupos)}',
                        'MB': f'{tamanio_mb:.1f}',
                        'imgs': len(grupo)
                    })

                logger.info(
                    f"  Parte {idx}/{len(grupos)}: {tamanio_mb:.1f} MB, "
                    f"{len(grupo)} imágenes"
                )

            except Exception as error:
                self.estadisticas.errores += 1
                logger.error(f"Error en {nombre_parte}: {error}")
                raise

        self.estadisticas.pdfs_generados += len(pdfs_generados)
        self.estadisticas.pdfs_con_partes += 1
        self.estadisticas.total_partes_generadas += len(pdfs_generados)

        logger.info(
            f"PDF dividido completado: {nombre_config} "
            f"({len(grupos)} partes, {len(rutas_imagenes)} imágenes)"
        )

        return pdfs_generados

    def generar_todos_los_pdfs(self, forzar_reinicio: bool = False) -> Dict[str, List[Path]]:
        """
        Genera todos los PDFs pendientes con checkpoints.

        Args:
            forzar_reinicio: Si True, ignora PDFs existentes y empieza desde cero

        Returns:
            Diccionario con configuración como clave y lista de PDFs como valor
        """
        print("\n" + "=" * 70)
        print("GENERACIÓN DE PDFs CON CONTROL DE PESO")
        print("=" * 70 + "\n")

        if forzar_reinicio:
            if self.checkpoint_file.exists():
                self._eliminar_checkpoint()
            self.configs_completadas = set()
            logger.info("Reinicio forzado: ignorando PDFs existentes")

        self.cargar_indice()
        self.agrupar_por_configuracion()
        configuraciones = self.extraer_configuraciones()

        if not configuraciones:
            logger.error("No se encontraron configuraciones")
            return {}

        configs_pendientes = sorted(configuraciones - self.configs_completadas)

        if not configs_pendientes:
            print("\n" + "=" * 70)
            print("¡TODAS LAS CONFIGURACIONES YA ESTÁN COMPLETAS!")
            print("=" * 70)
            print(f"\nTotal: {len(configuraciones)} configuraciones")
            print(f"Ubicación: {self.directorio_salida}")
            self._eliminar_checkpoint()
            return {}

        print(f"CONFIGURACIÓN DEL PROCESO:")
        print(f"  Total configuraciones:     {len(configuraciones):>4}")
        print(f"  Ya completadas (previas):  {len(self.configs_completadas):>4}")
        print(f"  Pendientes por generar:    {len(configs_pendientes):>4}")
        print(f"  Checkpoint cada:           {self.CHECKPOINT_INTERVALO:>4} PDFs")
        print(f"  Peso máximo por PDF:       {self.config_pdf.peso_maximo_mb:>4.1f} MB")
        if self.config_pdf.modelos_reprocesar:
            print(f"  Modelos seleccionados:     {', '.join(self.config_pdf.modelos_reprocesar)}")
        print()

        pdfs_generados: Dict[str, List[Path]] = {}
        errores: List[Tuple[str, str]] = []

        pbar_pdfs = tqdm(
            configs_pendientes,
            desc="Generando PDFs",
            unit="config",
            ncols=100,
            colour='green',
            initial=len(self.configs_completadas),
            total=len(configuraciones)
        )

        contador_desde_checkpoint = 0

        for config in pbar_pdfs:
            try:
                visualizaciones = self.agrupaciones[config]
                rutas_ordenadas = self.ordenar_visualizaciones(visualizaciones)

                rutas_pdfs = self.generar_pdf(config, rutas_ordenadas, pbar=pbar_pdfs)
                pdfs_generados[config] = rutas_pdfs
                self.configs_completadas.add(config)

                contador_desde_checkpoint += 1

                if contador_desde_checkpoint >= self.CHECKPOINT_INTERVALO:
                    self._guardar_checkpoint()
                    contador_desde_checkpoint = 0

            except Exception as error:
                errores.append((config, str(error)))
                continue

        pbar_pdfs.close()

        if contador_desde_checkpoint > 0:
            self._guardar_checkpoint()

        if len(self.configs_completadas) >= len(configuraciones):
            self._eliminar_checkpoint()

        self._mostrar_resumen(pdfs_generados, errores, len(configuraciones))

        return pdfs_generados

    def _mostrar_resumen(
        self,
        pdfs_generados: Dict[str, List[Path]],
        errores: List[Tuple[str, str]],
        total_configs: int
    ) -> None:
        """Muestra resumen detallado del proceso."""
        print("\n" + "=" * 70)
        print("RESUMEN DE GENERACIÓN")
        print("=" * 70 + "\n")

        print("ESTADÍSTICAS:")
        print("-" * 70)
        print(f"  Total configuraciones:           {total_configs:>6}")
        print(f"  Configs ya existentes (previas): {len(self.configs_completadas) - len(pdfs_generados):>6}")
        print(f"  Configs generadas esta sesión:   {len(pdfs_generados):>6}")
        print(f"  Configs totales completadas:     {len(self.configs_completadas):>6}")
        print(f"  PDFs generados (archivos):       {self.estadisticas.pdfs_generados:>6}")
        print(f"  PDFs con múltiples partes:       {self.estadisticas.pdfs_con_partes:>6}")
        print(f"  Total de partes generadas:       {self.estadisticas.total_partes_generadas:>6}")
        print(f"  Imágenes procesadas (sesión):    {self.estadisticas.total_imagenes_procesadas:>6}")
        print(f"  Errores durante generación:      {len(errores):>6}")
        print(f"  Warnings de canal alpha:         {self.estadisticas.alpha_warnings:>6}")

        if pdfs_generados:
            total_archivos_generados = sum(len(pdfs) for pdfs in pdfs_generados.values())
            tamanio_total_mb = sum(
                pdf.stat().st_size
                for pdfs in pdfs_generados.values()
                for pdf in pdfs
            ) / (1024 * 1024)
            promedio_mb = tamanio_total_mb / total_archivos_generados if total_archivos_generados > 0 else 0

            print(f"  Archivos PDF generados (total):  {total_archivos_generados:>6}")
            print(f"  Tamaño generado (sesión):        {tamanio_total_mb:>6.2f} MB")
            print(f"  Tamaño promedio por archivo:     {promedio_mb:>6.2f} MB")

        print("-" * 70)

        if errores:
            print(f"\nERRORES DETECTADOS ({len(errores)}):")
            for config, error in errores[:5]:
                print(f"  - {config}: {error[:60]}...")
            if len(errores) > 5:
                print(f"  ... y {len(errores) - 5} errores más")

        if self.estadisticas.alpha_warnings > 0:
            print(f"\nNOTA: {self.estadisticas.alpha_warnings} imágenes con canal alpha procesadas.")
            print("   (Normal en PNGs con transparencia)")

        if self.estadisticas.pdfs_con_partes > 0:
            print(f"\nNOTA: {self.estadisticas.pdfs_con_partes} configuraciones divididas en múltiples partes")
            print(f"   debido al límite de {self.config_pdf.peso_maximo_mb} MB por archivo")

        pendientes = total_configs - len(self.configs_completadas)
        if pendientes > 0:
            print(f"\nQUEDAN {pendientes} CONFIGURACIONES PENDIENTES")
            print("  Ejecute nuevamente esta celda para continuar")
        else:
            print("\n¡PROCESO COMPLETADO EXITOSAMENTE!")
            print(f"  Ubicación: {self.directorio_salida}")

        print("\n" + "=" * 70 + "\n")

In [7]:
# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================

def main(
    forzar_reinicio: bool = False,
    peso_maximo_mb: float = 15.0,
    modelos_reprocesar: Optional[List[str]] = None
) -> None:
    """
    Función principal para ejecutar la generación.

    Args:
        forzar_reinicio: Si True, ignora checkpoint y PDFs existentes
        peso_maximo_mb: Peso máximo permitido por PDF en megabytes
        modelos_reprocesar: Lista de modelos a reprocesar. Ejemplos:
            - None: procesa todos los modelos
            - ["mask2former"]: solo Mask2Former
            - ["mask2former", "bodypix"]: Mask2Former y BodyPix
            - ["yolo", "sam"]: YOLOv8-seg y SAM
    """
    print("Paso 4/4: Iniciando generación de PDFs con control de peso...\n")

    # Configuración de rutas del TFM
    RUTA_INDICE = "/content/drive/MyDrive/TFM/3_Analisis/fase1_integracion/indice_maestro.json"
    RUTA_SALIDA = "/content/drive/MyDrive/TFM/3_Analisis/00_Visualizaciones"

    try:
        # Crear configuración personalizada
        config_pdf = ConfiguracionPDF(
            peso_maximo_mb=peso_maximo_mb,
            modelos_reprocesar=modelos_reprocesar,
            prefijo_partes="parte"
        )

        generador = GeneradorPDFVisualizaciones(
            ruta_indice=RUTA_INDICE,
            directorio_salida=RUTA_SALIDA,
            config_pdf=config_pdf
        )

        pdfs_generados = generador.generar_todos_los_pdfs(forzar_reinicio=forzar_reinicio)

        if pdfs_generados:
            total_archivos = sum(len(pdfs) for pdfs in pdfs_generados.values())
            print(f"Sesión completada: {len(pdfs_generados)} configuraciones, "
                  f"{total_archivos} archivos PDF generados\n")

    except FileNotFoundError as error:
        print(f"\nERROR: Archivo no encontrado")
        print(f"  {error}")
        print(f"\nVerifique que las rutas sean correctas:")
        print(f"  Índice: {RUTA_INDICE}")
        print(f"  Salida: {RUTA_SALIDA}")
        raise

    except Exception as error:
        print(f"\nERROR INESPERADO: {error}")
        logger.exception("Traceback completo:")
        raise

In [10]:
if __name__ == "__main__":
    # Opción 1: Solo Mask2Former
    #main(modelos_reprocesar=["mask2former"])

    # Opción 2: Mask2Former y BodyPix
    main(modelos_reprocesar=["mask2former", "bodypix"])

Paso 4/4: Iniciando generación de PDFs con control de peso...


GENERACIÓN DE PDFs CON CONTROL DE PESO

CONFIGURACIÓN DEL PROCESO:
  Total configuraciones:       43
  Ya completadas (previas):     0
  Pendientes por generar:      43
  Checkpoint cada:              5 PDFs
  Peso máximo por PDF:       15.0 MB
  Modelos seleccionados:     mask2former, bodypix



Generando PDFs:   0%|                                                    | 0/43 [00:00<?, ?config/s]


RESUMEN DE GENERACIÓN

ESTADÍSTICAS:
----------------------------------------------------------------------
  Total configuraciones:               43
  Configs ya existentes (previas):      0
  Configs generadas esta sesión:       43
  Configs totales completadas:         43
  PDFs generados (archivos):           98
  PDFs con múltiples partes:           41
  Total de partes generadas:           96
  Imágenes procesadas (sesión):       793
  Errores durante generación:           0
  Warnings de canal alpha:            793
  Archivos PDF generados (total):      98
  Tamaño generado (sesión):        854.28 MB
  Tamaño promedio por archivo:       8.72 MB
----------------------------------------------------------------------

NOTA: 793 imágenes con canal alpha procesadas.
   (Normal en PNGs con transparencia)

NOTA: 41 configuraciones divididas en múltiples partes
   debido al límite de 15.0 MB por archivo

¡PROCESO COMPLETADO EXITOSAMENTE!
  Ubicación: /content/drive/MyDrive/TFM/3_Analis